In [1]:
# import jupyter notebook version of dash framework
from jupyter_dash import JupyterDash as Dash
# import dash components
from dash import Input, Output, State, html, dcc
# Import warnings to ignore warnings
import warnings
warnings.filterwarnings('ignore')
import os
from io import BytesIO
import numpy as np
from PIL import Image
import base64
# import load_model from keras
import tensorflow as tf
# Import visualization
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
# Import Explainer 
from lime import lime_image

# For now we will load it locally
model = tf.keras.models.load_model("../models/op_model1_aug.keras")

# Define style sheet 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Create the dash app
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Create markdown for our dashboard
markdown_1 = html.Div(dcc.Markdown("""
    # Brain Tumor Lesion Assesment
"""))

markdown_2 = html.Div(dcc.Markdown("""
    The model integrated in into this dashbaord has the capability to predict different brian lesions.
    The four supported brain lession classification are Meningioma, Pituitary, Glioma tumors. 
    The fourth possible prediction would be a No Tumor classification.
"""))

# Upload image
upload_img = dcc.Upload(
    id='upload-image',
    children=html.Div([
        'Drag and Drop or ',
        html.A('Select File')
    ]),
        style={
        'width': '100%',
        'height': '60px',
        'lineHeight': '60px',
        'borderWidth': '1px',
        'borderStyle': 'dashed',
        'borderRadius': '5px',
        'textAlign': 'center',
        'margin': '10px'
    },
    multiple=True, # Do not allow multiple uploads
)

# Define the output component
prediction_output = html.Div(id='prediction-output')
lime_mask_img = html.Div(id='lime-mask-container')
lime_heatmap_img = html.Div(id='heatmap-container')

# Set the layout
app.layout = html.Div(children=[
    markdown_1,
    markdown_2,
    html.Div([upload_img, html.Div(id='output-image-upload')]), # Upload image and update image
    prediction_output,
    lime_mask_img, # Display Lime Mask
    lime_heatmap_img, # Display Importance Heatmap
])


############################################################################################################################
# function to parse file path 
def parse_contents(contents, filename):
    return html.Div([
        html.H5(filename),

        # HTML images accept base64 encoded strings in the same format
        # that is supplied by the upload
        html.Img(src=contents),
        html.Hr()
    ])
# Define callback to change image upload
@app.callback(
    Output('output-image-upload', 'children'),
    Input('upload-image', 'contents'),
    State('upload-image', 'filename')
)
# Define an update function for the uploaded image
def update_output(list_of_contents, list_of_names):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n) for c, n in
            zip(list_of_contents, list_of_names)]
        return children 


# Define a function to generate Lime explanations
def generate_lime_explanation(image_bytes):
    if image_bytes is not None:
        # Load the Lime explainer
        explainer = lime_image.LimeImageExplainer(random_state=42)
        
        # Preprocess the image for Lime explanation and model prediction
        decoded_image = base64.b64decode(image_bytes.split(",")[1])
        img = Image.open(BytesIO(decoded_image))
        img = img.convert('RGB') # Convert image to RGB
        img = img.resize((128, 128)) # Resize the image to expected model image dimensions  
        img = np.array(img) / 255.0  # Normalize the image
        
        # Develop local model explanation
        explanation = explainer.explain_instance(
            image=img,
            classifier_fn=model.predict,
            top_labels=4,
            num_samples=2000,
            hide_color=0,
            random_seed=42
        )
        
        return explanation

# Define a function to display Lime explanations
def display_lime_explanation(explanation):
    # Obtain mask and image from the explainer
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],  # Using the top predicted label for visualization
        positive_only=True,
        num_features=5,
        hide_rest=True,
        min_weight=0.1
    )
    
    # Display the Lime Mask
    plt.figure(figsize=(8, 6), facecolor='white')
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.title("Concerning Area", fontsize=20)
    plt.axis("off")
    plt.tight_layout()
    
    # Save the Lime Mask Plot as bytes
    mask_buf = BytesIO()
    plt.savefig(mask_buf, format='png')
    mask_buf.seek(0)
    mask_bytes64 = base64.b64encode(mask_buf.read()).decode()
    plt.close()
    
    # Obtain heatmap version that explains the areas that contribute most to that decision
    ind = explanation.top_labels[0]
    dict_heatmap = dict(explanation.local_exp[ind])
    heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
    
    # Display the heatmap
    plt.figure(figsize=(8, 6), facecolor='white')
    plt.imshow(heatmap, cmap='RdBu', vmin=-heatmap.max(), vmax=heatmap.max())
    plt.title("Blue = More Important; Red = Less Important", fontsize=20)
    plt.axis("off")  # Hide axes
    plt.tight_layout()
    
    # Save the Heatmap plot as bytes
    heatmap_buf = BytesIO()
    plt.savefig(heatmap_buf, format='png')
    heatmap_buf.seek(0)
    heatmap_bytes64 = base64.b64encode(heatmap_buf.read()).decode()
    plt.close()
    
    return mask_bytes64, heatmap_bytes64

# Define the callback to preprocess the image and make predictions
@app.callback(
    [Output('prediction-output', 'children'),
    Output('lime-mask-container', 'children'),
    Output('heatmap-container', 'children')],
    [Input('upload-image', 'contents')]
)

def update_prediction_output(contents):
    if contents is not None:
        max_prediction_label = None
        max_prediction_value = 0
        lime_mask_images = []
        heatmap_images = []
        
        for content in contents:
            content_type, content_string = content.split(',')
            
            # Decode the uploaded image
            decoded_image = base64.b64decode(content_string)
            
            # Preprocess the image
            img = Image.open(BytesIO(decoded_image))
            img = img.convert('RGB') # Convert image to RGB
            img = img.resize((128, 128)) # Resize the image to expected model image dimensions  
            img = np.array(img) / 255.0  # Normalize the image
            
            # Make prediction
            prediction = model.predict(np.expand_dims(img, axis=0))
            
            # Get the index of the class with the highest probability
            max_index = np.argmax(prediction)
            
            # Map the index to the corresponding class label
            if max_index == 0:
                max_prediction_label = 'glioma'
            elif max_index == 1:
                max_prediction_label = 'meningioma'
            elif max_index == 2:
                max_prediction_label = 'no_tumor'
            elif max_index == 3:
                max_prediction_label = 'pituitary'
            
            # If the max probability for this image is higher, update the label
            if prediction[0, max_index] > max_prediction_value:
                max_prediction_value = prediction[0, max_index]
                
            # Generate Lime explanation
            explanation = generate_lime_explanation(content_string)
                
            # Get Lime bytes for image_mask and heatmap  
            lime_mask_bytes, heatmap_bytes = display_lime_explanation(explanation)
            lime_mask_images.append('data:image/png;base64,' + lime_mask_bytes)
            heatmap_images.append('data:image/png;base64,' + heatmap_bytes)
            
        # Return label with highest probablity 
        if max_prediction_label:
            return f"Prediction: {max_prediction_label.capitalize()}", lime_mask_images, heatmap_images
        else:
            return "No prediction available", [], []

##############################################################################################################################
if __name__ == '__main__':
    app.run_server(mode='inline', host='localhost', port=5000)